In [1]:
from Bio import SeqIO
import pandas as pd
import numpy as np


chrdict = {}

for i, record in enumerate(SeqIO.parse("..\\dataset\\\GRCh37.primary_assembly.genome.fa", "fasta")):
    chrdict[record.id] = record.seq


# LE SE SEQUENZE DI BASI AZOTATE SONO MEMORIZZATE IN UN DIZIONARIO CHE USA COME CHIAVE "chr#" dove # indica
# il numero del cromosoma oppure X/Y/M (M non ci interessa perché é il DNA mitocondriale che non viene usato per questo caso)

<>:8: SyntaxWarning: invalid escape sequence '\G'
<>:8: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Danilo\AppData\Local\Temp\ipykernel_13416\4216738839.py:8: SyntaxWarning: invalid escape sequence '\G'
  for i, record in enumerate(SeqIO.parse("..\\dataset\\\GRCh37.primary_assembly.genome.fa", "fasta")):


In [8]:
chrdict

{'chr1': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr10': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr11': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr12': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr13': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr14': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr15': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr16': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr17': Seq('AAGCTTCTCACCCTGTTCCTGCATAGATAATTGCATGACAATTGCCTTGTCCCT...GGT'),
 'chr18': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr19': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr2': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'),
 'chr20': Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN

In [30]:
# UA INIZIA INVECE LA PARTE DI LETTURA DEL FILE CONTENENTE I TSS

from Bio import SeqIO
import pandas as pd
import numpy as np
def extract_p(p):
    return p.split("@")[0]

def extract_gene_name(string):
    string =  string.split("@")[1]
    return string.split(",")[0]

v1 = np.vectorize(extract_p)
v2 = np.vectorize(extract_gene_name)

df = pd.read_csv("..\\dataset\\TSS_human.bed", delimiter="\t", skiprows=1, names=["chromosome_name", "start", "end", "locus_id", "color", "strand", "start2", "end2", "RGB"])



# Applica la funzione alla colonna del DataFrame
df['p'] = v1(df['locus_id'].values)
df['gene_name'] = v2(df['locus_id'].values)
df = df.drop(columns=["locus_id", "color", "start2", "end2", "RGB"])

df

,chromosome_name,start,end,strand,p,gene_name
0,chr10,100008587,100008589,+,p1,CU680531
1,chr10,100015362,100015397,-,p2,LOXL4
2,chr10,100017518,100017519,-,p3,LOXL4
3,chr10,100027943,100027958,-,p1,LOXL4
4,chr10,100174900,100174956,-,p1,PYROXD2
...,...,...,...,...,...,...
1048119,chrY,9958814,9958815,-,p,chrY:9958814..9958815
1048120,chrY,9959226,9959229,-,p,chrY:9959226..9959229
1048121,chrY,9968102,9968103,-,p,chrY:9968102..9968103
1048122,chrY,9985705,9985711,-,p,chrY:9985705..9985711


In [31]:
import re

def estrai_numero(s):
    match = re.match(r'p(\d+)', s)
    if match:
        return int(match.group(1))
    else:
        return None


df['p_value'] = df['p'].apply(estrai_numero)
#droppa righe con p_value non definito
df = df.dropna()
df

,chromosome_name,start,end,strand,p,gene_name,p_value
0,chr10,100008587,100008589,+,p1,CU680531,1.0
1,chr10,100015362,100015397,-,p2,LOXL4,2.0
2,chr10,100017518,100017519,-,p3,LOXL4,3.0
3,chr10,100027943,100027958,-,p1,LOXL4,1.0
4,chr10,100174900,100174956,-,p1,PYROXD2,1.0
...,...,...,...,...,...,...,...
294760,chrY,7142165,7142189,+,p3,PRKY,3.0
294761,chrY,7142206,7142215,+,p9,PRKY,9.0
294762,chrY,7200834,7200838,+,p12,PRKY,12.0
294763,chrY,7200913,7200914,+,p14,PRKY,14.0


In [32]:
df = df.loc[df.groupby("gene_name")["p_value"].idxmin()]
df.drop(columns=["p", "p_value"], inplace=True)
df

,chromosome_name,start,end,strand,gene_name
65039,chr19,58858938,58859039,-,A1BG
65040,chr19,58859101,58859149,+,A1BG-AS1
4033,chr10,52645379,52645393,-,A1CF
27660,chr13,101240987,101241049,-,A2LD1
27124,chr12,9268507,9268523,-,A2M
...,...,...,...,...,...
287241,chr9,131012693,131012727,-,uc011mav.1
159387,chr9,135895676,135895723,+,uc011mct.1
288056,chr9,138899746,138899753,-,uc011mds.1
290683,chrM,6330,6392,+,uc011mfh.1


In [34]:
lizio=pd.read_csv("..\\dataset\\Join_with_Lizio.csv",sep=",")
lizio.drop(columns=["Unnamed: 0"],inplace=True)
lizio

,gene_name,gene_stable_id,gene_id,gene_type,TSS,strand,chromosome_name
0,A1BG,ENSG00000121410,ENSG00000121410.12,protein_coding,58347673,-,chr19
1,A1CF,ENSG00000148584,ENSG00000148584.15,protein_coding,50885633,-,chr10
2,A2M,ENSG00000175899,ENSG00000175899.15,protein_coding,9115927,-,chr12
3,A2ML1,ENSG00000166535,ENSG00000166535.20,protein_coding,8822627,+,chr12
4,A4GALT,ENSG00000128274,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...,...
15615,ZYG11A,ENSG00000203995,ENSG00000203995.10,protein_coding,52842770,+,chr1
15616,ZYG11B,ENSG00000162378,ENSG00000162378.13,protein_coding,52726478,+,chr1
15617,ZYX,ENSG00000159840,ENSG00000159840.16,protein_coding,143381361,+,chr7
15618,ZZEF1,ENSG00000074755,ENSG00000074755.15,protein_coding,4143055,-,chr17


In [35]:
#join dei due dataframe su gene_name
df2 = pd.merge(df, lizio, on="gene_name")
df2

,chromosome_name_x,start,end,strand_x,gene_name,gene_stable_id,gene_id,gene_type,TSS,strand_y,chromosome_name_y
0,chr19,58858938,58859039,-,A1BG,ENSG00000121410,ENSG00000121410.12,protein_coding,58347673,-,chr19
1,chr10,52645379,52645393,-,A1CF,ENSG00000148584,ENSG00000148584.15,protein_coding,50885633,-,chr10
2,chr12,9268507,9268523,-,A2M,ENSG00000175899,ENSG00000175899.15,protein_coding,9115927,-,chr12
3,chr12,8975206,8975223,+,A2ML1,ENSG00000166535,ENSG00000166535.20,protein_coding,8822627,+,chr12
4,chr22,43116867,43116885,-,A4GALT,ENSG00000128274,ENSG00000128274.17,protein_coding,42720879,-,chr22
...,...,...,...,...,...,...,...,...,...,...,...
15615,chr1,53308425,53308442,+,ZYG11A,ENSG00000203995,ENSG00000203995.10,protein_coding,52842770,+,chr1
15616,chr1,53192114,53192150,+,ZYG11B,ENSG00000162378,ENSG00000162378.13,protein_coding,52726478,+,chr1
15617,chr7,143078379,143078454,+,ZYX,ENSG00000159840,ENSG00000159840.16,protein_coding,143381361,+,chr7
15618,chr17,4046257,4046349,-,ZZEF1,ENSG00000074755,ENSG00000074755.15,protein_coding,4143055,-,chr17


In [60]:
df2=df2[df2['strand_x']==df2['strand_y']]
df2.drop(columns=["strand_y"],inplace=True)

C:\Users\Danilo\AppData\Local\Temp\ipykernel_10648\3414141631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=["strand_y"],inplace=True)


In [61]:
meging=pd.read_csv("..\\dataset\\ens37_mancanti.txt", sep="\t")
meging

,gene_id,Transcription start site (TSS),Chromosome/scaffold name,strand,gene_name,Gene type
0,ENSG00000261657.1,66119285,HG991_PATCH,1,SLC25A26,protein_coding
1,ENSG00000261657.1,66320895,HG991_PATCH,1,SLC25A26,protein_coding
2,ENSG00000261657.1,66320895,HG991_PATCH,1,SLC25A26,protein_coding
3,ENSG00000261657.1,66320895,HG991_PATCH,1,SLC25A26,protein_coding
4,ENSG00000261657.1,66339287,HG991_PATCH,1,SLC25A26,protein_coding
...,...,...,...,...,...,...
215399,ENSG00000263267.1,198858,HSCHR17_1_CTG1,1,RP11-1260E13.4,processed_transcript
215400,ENSG00000263267.1,199139,HSCHR17_1_CTG1,1,RP11-1260E13.4,processed_transcript
215401,ENSG00000263267.1,199766,HSCHR17_1_CTG1,1,RP11-1260E13.4,processed_transcript
215402,ENSG00000262336.1,220262,HSCHR17_1_CTG1,1,RP11-1260E13.2,antisense


In [62]:
#modifica la colonna Strand assegnando - se e uguale a -1 e + se e uguale a 1
meging['strand'] = meging['strand'].apply(lambda x: "-" if x == -1 else "+")
meging

,gene_id,Transcription start site (TSS),Chromosome/scaffold name,strand,gene_name,Gene type
0,ENSG00000261657.1,66119285,HG991_PATCH,+,SLC25A26,protein_coding
1,ENSG00000261657.1,66320895,HG991_PATCH,+,SLC25A26,protein_coding
2,ENSG00000261657.1,66320895,HG991_PATCH,+,SLC25A26,protein_coding
3,ENSG00000261657.1,66320895,HG991_PATCH,+,SLC25A26,protein_coding
4,ENSG00000261657.1,66339287,HG991_PATCH,+,SLC25A26,protein_coding
...,...,...,...,...,...,...
215399,ENSG00000263267.1,198858,HSCHR17_1_CTG1,+,RP11-1260E13.4,processed_transcript
215400,ENSG00000263267.1,199139,HSCHR17_1_CTG1,+,RP11-1260E13.4,processed_transcript
215401,ENSG00000263267.1,199766,HSCHR17_1_CTG1,+,RP11-1260E13.4,processed_transcript
215402,ENSG00000262336.1,220262,HSCHR17_1_CTG1,+,RP11-1260E13.2,antisense


In [63]:
merged = pd.merge(df2, meging, on="gene_id", how="inner")
merged

,chromosome_name_x,start,end,strand_x,gene_name_x,gene_stable_id,gene_id,gene_type,TSS,chromosome_name_y,Transcription start site (TSS),Chromosome/scaffold name,strand,gene_name_y,Gene type
0,chr14,64932210,64932226,+,AKAP5,ENSG00000179841,ENSG00000179841.8,protein_coding,64465508,chr14,64932217,14,+,AKAP5,protein_coding
1,chr14,64932210,64932226,+,AKAP5,ENSG00000179841,ENSG00000179841.8,protein_coding,64465508,chr14,64933825,14,+,AKAP5,protein_coding
2,chr10,124214164,124214184,+,ARMS2,ENSG00000254636,ENSG00000254636.1,protein_coding,122454668,chr10,124214169,10,+,ARMS2,protein_coding
3,chr20,3065362,3065375,-,AVP,ENSG00000101200,ENSG00000101200.5,protein_coding,3084729,chr20,3065370,20,-,AVP,protein_coding
4,chr19,41933888,41933907,-,B3GNT8,ENSG00000177191,ENSG00000177191.2,protein_coding,41428002,chr19,41934635,19,-,B3GNT8,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,chr16,31085714,31085783,+,ZNF646,ENSG00000167395,ENSG00000167395.10,protein_coding,31074462,chr16,31087223,16,+,ZNF646,protein_coding
270,chr15,35280439,35280503,-,ZNF770,ENSG00000198146,ENSG00000198146.4,protein_coding,34988302,chr15,35280488,15,-,ZNF770,protein_coding
271,chr15,35280439,35280503,-,ZNF770,ENSG00000198146,ENSG00000198146.4,protein_coding,34988302,chr15,35280426,15,-,ZNF770,protein_coding
272,chr10,126669977,126669982,+,ZRANB1,ENSG00000019995,ENSG00000019995.6,protein_coding,124981413,chr10,126630692,10,+,ZRANB1,protein_coding


In [4]:
patient=pd.read_csv("..\\dataset\\Dataset_median.csv", sep=",")
#merge su gene_id
df2 = pd.merge(df1, patient, on="gene_id")
df2

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median
0,chr10,100015362,100015397,"p2@LOXL4,0.1291",55.0,-,100015362,100015397,"211,211,211",p2,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
1,chr10,100017518,100017519,"p3@LOXL4,0.1842",13.0,-,100017518,100017519,"30,144,255",p3,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
2,chr10,100027943,100027958,"p1@LOXL4,0.2200",48.0,-,100027943,100027958,"30,144,255",p1,LOXL4,ENSG00000138131.4,1.800827,3.202559,1.977610
3,chr10,100174900,100174956,"p1@PYROXD2,0.2721",0.0,-,100174900,100174956,"60,179,113",p1,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999
4,chr10,100174957,100174982,"p2@PYROXD2,0.2448",0.0,-,100174957,100174982,"60,179,113",p2,PYROXD2,ENSG00000119943.13,1.167486,2.608927,1.382999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153925,chrY,2870826,2870844,"p2@ZFY-AS1,0.0950",-159.0,-,2870826,2870844,"211,211,211",p2,ZFY-AS1,ENSG00000233070.1,-7.643856,-7.643856,-7.643856
153926,chrY,6778505,6778534,"p6@TBL1Y,0.3422",-192.0,+,6778505,6778534,"60,179,113",p6,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856
153927,chrY,6778592,6778601,"p7@TBL1Y,0.4173",-125.0,+,6778592,6778601,"60,179,113",p7,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856
153928,chrY,6778602,6778616,"p4@TBL1Y,0.4601",-110.0,+,6778602,6778616,"60,179,113",p4,TBL1Y,ENSG00000092377.15,-7.643856,-7.643856,-7.643856


In [9]:
df2["gene_name"].value_counts()


gene_name
MEG3         176
SLC1A2       170
PAPPA        153
APOB         147
QKI          137
            ... 
PAPPA-AS1      1
MRPL50P2       1
RPS26P49       1
KRT8P15        1
ANKRD36P1      1
Name: count, Length: 14198, dtype: int64

In [28]:

df3 = df2.loc[df2.groupby("gene_name")["p_value"].idxmin()]



df3.value_counts("p")

p
p1    14075
p2       86
p3       14
p4       10
p5        5
p6        3
p7        2
p8        2
p9        1
Name: count, dtype: int64

In [27]:
df4 = df2.loc[df2.groupby("gene_id")["p_value"].idxmin()]
df4

,chromosome_name,start,end,locus_id,color,strand,start2,end2,RGB,p,gene_name,gene_id,fpkm_median,tpm_median,fpkm_uq_median,p_value
153866,chrX,99839925,99839938,"p1@TNMD,0.1353",126.0,+,99839925,99839938,"211,211,211",p1,TNMD,ENSG00000000005.6,-3.925769,-2.475535,-3.733124,1
38903,chr1,169863016,169863109,"p1@SCYL3,0.2639",0.0,-,169863016,169863109,"60,179,113",p1,SCYL3,ENSG00000000457.14,0.873892,2.396543,1.063089,1
42198,chr1,27961629,27961665,"p1@FGR,0.5037",0.0,-,27961629,27961665,"60,179,113",p1,FGR,ENSG00000000938.13,0.612022,2.081715,0.757621,1
71796,chr6,143832820,143832875,"p1@FUCA2,0.4000",0.0,-,143832820,143832875,"60,179,113",p1,FUCA2,ENSG00000001036.14,3.827545,5.333456,4.001946,1
74636,chr6,53409890,53409906,"p1@GCLC,0.4228",-4.0,-,53409890,53409906,"60,179,113",p1,GCLC,ENSG00000001084.13,1.509544,2.956224,1.694702,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86592,chrX,101967366,101967391,"p1@ARMCX5-GPRASP2,p1@GPRASP2,0.6351",0.0,+,101967366,101967391,"60,179,113",p1,ARMCX5-GPRASP2,ENSG00000286237.1,-7.643856,-7.643856,-7.643856,1
65961,chr4,89513578,89513670,"p1@HERC3,0.3732",0.0,+,89513578,89513670,"60,179,113",p1,HERC3,ENSG00000287542.1,-7.643856,-7.643856,-7.643856,1
82689,chr8,67579807,67579860,"p1@C8orf44,p1@SGK3,0.4647",0.0,+,67579807,67579860,"60,179,113",p1,C8orf44,ENSG00000288596.2,0.995448,2.458933,1.200316,1
148900,chr8,53852965,53852990,"p1@NPBWR1,0.2996",497.0,+,53852965,53852990,"60,179,113",p1,NPBWR1,ENSG00000288611.1,-6.687800,-5.779918,-6.546245,1
